In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# One cell RNN input_dim (3) -> output_dim (5)
hidden_size = 5
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
print(cell.output_size, cell.state_size)

x_data = np.array([[[1, 2, 3]]], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

5 5
array([[[ 0.75732291, -0.13325819, -0.52792329,  0.4675394 , -0.16449949]]], dtype=float32)


In [3]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# One cell RNN input_dim (3) -> output_dim (5). sequence: 2
hidden_size = 5
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]]], dtype=np.float32)
outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[ 0.93098164,  0.33355391, -0.39505908, -0.96330673,  0.77468497],
        [ 0.99974811,  0.76130593, -0.68241394, -0.99999744,  0.97888589]]], dtype=float32)


In [4]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# One cell RNN input_dim (3) -> output_dim (5). sequence: 2
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]],

                   [[7, 8, 9],
                    [10, 11, 12]],

                   [[13, 14, 15],
                    [16, 17, 18]], ], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_data, sequence_length=[1,2,1], dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[ -2.28231046e-02,  -2.78009206e-01,  -2.17054889e-01,
          -1.07388906e-01,  -3.30501080e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -6.72279019e-03,  -2.42554191e-02,  -7.11063087e-01,
          -2.40728557e-01,  -6.72493994e-01],
        [ -1.63571001e-03,  -1.13375830e-02,  -9.30618346e-01,
          -2.49628484e-01,  -8.40059042e-01]],

       [[ -3.34159529e-04,  -1.14610826e-03,  -7.57456303e-01,
          -1.53216466e-01,  -7.46558070e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32)


In [5]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# One cell RNN input_dim (3) -> output_dim (5). sequence: 2

batch_size = 3
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)

x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]],

                   [[7, 8, 9],
                    [10, 11, 12]],

                   [[13, 14, 15],
                    [16, 17, 18]], ], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                     initial_state=initial_state, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[ -9.06838924e-02,   2.69779898e-02,  -4.16334607e-02,
           3.73940505e-02,  -7.32717663e-02],
        [ -6.04101196e-02,   1.83868650e-02,  -2.40339916e-02,
           9.85701606e-02,  -1.32940169e-02]],

       [[ -1.27240922e-02,   2.60907982e-04,  -7.46334917e-05,
           4.22762781e-02,  -1.22250535e-03],
        [ -4.69082315e-03,   7.36236761e-05,  -3.77873548e-05,
           1.57461427e-02,  -1.51262691e-04]],

       [[ -8.47805117e-04,   1.24200938e-06,  -7.01261698e-08,
           5.31122321e-03,  -1.42158951e-05],
        [ -2.73798796e-04,   1.51486347e-07,  -3.42364359e-08,
           1.81897194e-03,  -1.57338479e-06]]], dtype=float32)


In [6]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# Create input data
x_data = np.arange(24, dtype=np.float32).reshape(2, 4, 3)
pp.pprint(x_data)  # batch, sequence_length, input size

# Make rnn
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True)

# rnn in/out
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
print("dynamic rnn: ", outputs)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                     sequence_length=[1, 2])
print("dynamic rnn: ", outputs)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size
sess.close()

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.]],

       [[ 12.,  13.,  14.],
        [ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.]]], dtype=float32)
dynamic rnn:  Tensor("rnn/transpose:0", shape=(2, 4, 5), dtype=float32)
array([[[-0.00518472,  0.0035151 , -0.00511228, -0.00444861,  0.00375908],
        [-0.01759234,  0.0091941 , -0.01396412, -0.01754779,  0.01152858],
        [-0.03402757,  0.01297162, -0.02286568, -0.03665255,  0.02295469],
        [-0.05056015,  0.01167784, -0.02843066, -0.05681353,  0.03636513]],

       [[-0.00779046, -0.00297874,  0.00117829, -0.00724645, -0.00128889],
        [-0.02054391, -0.00929413,  0.00216497, -0.01936657, -0.00098175],
        [-0.03492529, -0.01857489,  0.00275036, -0.0331241 ,  0.00198329],
        [-0.04897475, -0.02997419,  0.00287493, -0.04629733,  0.00759039]]], dtype=float32)
dynamic rnn:  Tensor("rnn_1/transpose:0", shape=(2, 4, 5), dtype=f

In [7]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# bi-directional rnn
cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                  sequence_length=[2, 3],
                                                  dtype=tf.float32)

sess.run(tf.global_variables_initializer())
pp.pprint(sess.run(outputs))
pp.pprint(sess.run(states))

# Broadcasting based softmax
softmax_w = np.arange(12, dtype=np.float32).reshape(4, 3)
outputs = x_data * softmax_w
pp.pprint(softmax_w)
pp.pprint(outputs)
outputs = x_data * softmax_w + [1, 2, 3]
pp.pprint(outputs)

(   array([[[  2.95027614e-01,  -2.23175213e-01,  -1.49507802e-02,
           1.31392568e-01,  -2.29365360e-02],
        [  6.17041051e-01,  -5.78767538e-01,  -5.69266118e-02,
           1.85829490e-01,   1.03869941e-03],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  2.77294833e-02,  -6.30361289e-02,  -5.74521348e-03,
           1.45536178e-05,   2.89994802e-07],
        [  3.79111692e-02,  -9.33482647e-02,  -7.62500660e-03,
           1.57816248e-05,   2.75642869e-08],
        [  4.16267812e-02,  -1.07555307e-01,  -8.18355009e-03,
           1.58844177e-05,   2.06733497e-09],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32),
    array([[[  2.34663486e-01,   2.69046277e-02,  -2.62851357e-01,
           1.79598570e-01,  -3.481397

In [8]:
# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0, 1], [0, 1], [0, 1]]], dtype=tf.float32)
prediction2 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction3 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval())

Loss1:  0.313262 Loss2:  1.31326 Loss3:  0.646595
